In [7]:
import pickle

# Load variables from .pkl files
with open(r"/home/rvce/Downloads/HPCC/12k Data/all_symptoms_ALL.pkl", 'rb') as f:
    all_symptoms_ALL = pickle.load(f)

with open(r"/home/rvce/Downloads/HPCC/12k Data/symptom_with_organ_ALL.pkl", 'rb') as f:
    symptom_with_organ_ALL = pickle.load(f)

with open(r"/home/rvce/Downloads/HPCC/12k Data/symptom_with_duration_ALL.pkl", 'rb') as f:
    symptom_with_duration_ALL = pickle.load(f)

with open(r"/home/rvce/Downloads/HPCC/12k Data/pred_1_shuffled.pkl", 'rb') as f:
    pred_1_shuffled = pickle.load(f)

with open(r"/home/rvce/Downloads/HPCC/12k Data/feature_matrices.pkl", 'rb') as f:
    feature_matrices = pickle.load(f)

with open(r"/home/rvce/Downloads/HPCC/12k Data/edge_list_all.pkl", 'rb') as f:
    edge_list_all = pickle.load(f)


/home/rvce/durgarao/apps/miniconda3/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [8]:
print(len(all_symptoms_ALL))
print(len(symptom_with_organ_ALL))
print(len(pred_1_shuffled))
print(len(symptom_with_duration_ALL))
print(len(feature_matrices))
print(len(edge_list_all))


12000
12000
12000
12000
12000
12000


In [12]:
c=0
for i in range(len(feature_matrices)):
    c+=len(feature_matrices[i])

c/12000

14.65025

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SortAggregation
from torch_geometric.data import Data
import torch.optim as optim

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GCN_SortPool_CNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, k):
        super(GCN_SortPool_CNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        #self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.sort_pool = SortAggregation(k=k) #prioritizing nodes (dk what k is)
        self.cnn1d = nn.Conv1d(in_channels=hidden_channels, out_channels=32, kernel_size=2)
        
        # Calculate the correct input size for the fully connected layer
        cnn_output_size = 32 * (k - 1)  # Because kernel_size=2 reduces length by 1
        self.fc = nn.Linear(cnn_output_size, out_channels)
        
    def forward(self, x, edge_index):
        # GCN Layers
        x = F.relu(self.conv1(x, edge_index))
        #x = F.relu(self.conv2(x, edge_index))
        
        # SortPooling
        x = self.sort_pool(x, torch.zeros(x.size(0), dtype=torch.long, device=x.device))  # Create a dummy batch tensor
        
        # Reshape for 1D CNN
        x = x.view(x.size(0), -1, self.sort_pool.k)  # Reshape to (batch_size, hidden_channels, k)
        
        # 1D CNN
        x = F.relu(self.cnn1d(x))
        x = x.view(x.size(0), -1)  # Flatten
        
        # Fully Connected + Sigmoid
        x = self.fc(x)
        x = torch.sigmoid(x)  # Apply sigmoid activation
        return x

n = 12000  # Example number of graphs # change after skipping1500
# Define your model
model = GCN_SortPool_CNN(in_channels=768, hidden_channels=32, out_channels=1, k=14).to(device)  # Move model to device

# Initialize optimizer and criterion
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification

# Assuming you have lists of feature matrices and edge lists with varying node counts


# Create a list of Data objects
graphs = [Data(x=feature_matrices[i].to(device), edge_index=edge_list_all[i].to(device)) for i in range(n)]  # Move data to device

# Training loop
# pred_1 += pred_2
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i, graph in enumerate(graphs):
        optimizer.zero_grad()
        output = model(graph.x, graph.edge_index.view(2, -1))
        target = torch.tensor(pred_1_shuffled[i], dtype=torch.float32, device=device).view(1, 1)  # Move target to device
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(graphs)}")


Epoch 1, Average Loss: 0.5868396264809417
Epoch 2, Average Loss: 0.5453462819168781
Epoch 3, Average Loss: 0.5351913695823799
Epoch 4, Average Loss: 0.5290083462915863
Epoch 5, Average Loss: 0.524867566269438
Epoch 6, Average Loss: 0.5171086838761306
Epoch 7, Average Loss: 0.5100157206965426
Epoch 8, Average Loss: 0.5026475585501854
Epoch 9, Average Loss: 0.5017772803819631
Epoch 10, Average Loss: 0.4986044243983924
Epoch 11, Average Loss: 0.4947710732349757
Epoch 12, Average Loss: 0.49541939175699373
Epoch 13, Average Loss: 0.49474866045457505
Epoch 14, Average Loss: 0.4898237315934554
Epoch 15, Average Loss: 0.4902241719450734
Epoch 16, Average Loss: 0.4859061021190003
Epoch 17, Average Loss: 0.48441403933163385
Epoch 18, Average Loss: 0.480758275340823
Epoch 19, Average Loss: 0.48269684094916376
Epoch 20, Average Loss: 0.47813292360744286
Epoch 21, Average Loss: 0.48050299606461583
Epoch 22, Average Loss: 0.4805575003309835
Epoch 23, Average Loss: 0.47700002721027823
Epoch 24, Avera

In [16]:
import torch
torch.save(model.state_dict(), 'gcn_sortpool_cnn.pth')

In [15]:
print(len(edge_list_all[0]))
len(feature_matrices[0])

20


18

In [27]:
edge_list_all[1].view(2,-1).shape
# feature_matrices[0].shape


torch.Size([2, 27])